<span class="badge badge-info">ПРЕДОБРАБОТКА ДАННЫХ</span>
# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита

## Инструкция по выполнению проекта

**Шаг 1. Получение данных**
 - 1.1. Импорт библиотек и загрузка данных
 - 1.2. Осмотр данных
 - 1.3. Выводы

**Шаг 2. Предобработка данных**
 - 2.1. Замена названия столбцов
 - 2.2. Обработка пропусков
 - 2.3. Замена типа данных
 - 2.4. Преобразование строки в нижний регистр
 - 2.5. Обработка дубликатов
 - 2.6. Лемматизация
 - 2.7. Категоризация данных
 - 2.8. Выводы
 
**Шаг 3. Ответы на вопросы**
 - 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
 - 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
 - 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 - 3.4. Как разные цели кредита влияют на его возврат в срок?

**Шаг 4. Общий вывод исследования**

---

## Шаг 1. Получение данных

### 1.1. Импорт библиотек и загрузка данных

In [1]:
# Необходимые библиотеки.
import pandas as pd                      # для анализа данных
import numpy as np                       # для математических вычислений

#!pip install pymystem3
from pymystem3 import Mystem             # для лемматизации
from collections import Counter          # для подсчёта числа лемматизированных слов  

# Прочитаем файл credit_scoring.csv методом read_csv(), сохраним его в переменной df.
df = pd.read_csv('credit_scoring.csv')  

In [2]:
# С помощью pd.set_option настроим вывод так чтобы:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)                 # текст в ячейке отражался полностью вне зависимости от длины;
pd.set_option('display.float_format', '{:.2f}'.format)      # все числа отражались с двумя знаками после запятой.

### 1.2. Осмотр данных

Для знакомства с данными запросим несколько строк из начала и конца таблицы, вызывая методы head() и tail().

In [3]:
df.head(10)  # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу
5,0,-926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,покупка жилья
6,0,-2879.20,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем
7,0,-152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.93,образование
8,2,-6929.87,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.83,на проведение свадьбы
9,0,-2188.76,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,покупка жилья для семьи


In [4]:
df.tail(10) # получение последних 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.69,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.33,заняться образованием
21516,0,-914.39,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.78,покупка своего жилья
21517,0,-404.68,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.55,на покупку своего автомобиля
21518,0,373995.71,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.65,сделка с автомобилем
21519,1,-2351.43,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.04,покупка коммерческой недвижимости
21520,1,-4529.32,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.86,операции с жильем
21521,0,343937.40,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.81,сделка с автомобилем
21522,1,-2113.35,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.56,недвижимость
21523,3,-3112.48,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.05,на покупку своего автомобиля
21524,2,-1984.51,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.42,на покупку автомобиля


In [5]:
df.describe().T # выведем статистику о датасете методом describe()

,count,mean,std,min,25%,50%,75%,max
children,21525.00,0.54,1.38,-1.00,0.00,0.00,1.00,20.00
days_employed,19351.00,63046.50,140827.31,-18388.95,-2747.42,-1203.37,-291.10,401755.40
dob_years,21525.00,43.29,12.57,0.00,33.00,42.00,53.00,75.00
education_id,21525.00,0.82,0.55,0.00,1.00,1.00,1.00,4.00
family_status_id,21525.00,0.97,1.42,0.00,0.00,0.00,1.00,4.00
debt,21525.00,0.08,0.27,0.00,0.00,0.00,0.00,1.00
total_income,19351.00,167422.30,102971.57,20667.26,103053.15,145017.94,203435.07,2265604.03


In [6]:
df.describe(include='object').T # теперь включая только строковые столбцы 

,count,unique,top,freq
education,21525,15,среднее,13750
family_status,21525,5,женат / замужем,12380
gender,21525,3,F,14236
income_type,21525,8,сотрудник,11119
purpose,21525,38,свадьба,797


In [7]:
df.isnull().sum().sort_values() # суммарное количество пропусков, отсортируем методом sort_values()

children               0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
purpose                0
days_employed       2174
total_income        2174
dtype: int64

In [8]:
df.duplicated().sum() # количество дубликатов

54

In [9]:
df.info() # получение общей информации о данных в таблице df. Применим к таблице df метод info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 21525 строк (наблюдений) и 12 столбцов, в них встречаются разные типы данных: float64 (вещественные числа), int64 (целые числа) и object (строки). Видим, что в разных столбцах разное количество элементов с определёнными значениями. Следовательно, в таблице есть пропущенные значения, их нужно обработать.

### Вывод

Наблюдаются следующие проблемы с данными:

- названия столбцов написаны не совсем удобно и понятно;
- пропущенные значения в столбце 'общий трудовой стаж в днях' и 'ежемесячный доход';
- наличие дубликатов в переменных;
- аномальные значения: 
    отрицательные значения в переменных 'количество детей в семье' и 'общий трудовой стаж в днях', 
    нулевые значения в переменной 'возраст клиента в годах';
- вещественный тип данных в переменных 'общий трудовой стаж в днях' и 'ежемесячный доход';
- варьируется регистр букв в переменной 'уровень образования клиента'.

## Шаг 2. Предобработка данных

### 2.1. Замена названия столбцов

In [10]:
# Получаем перечень названий столбцов таблицы df.
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [11]:
# Переименуем некоторые столбцы для удобства дальнейшей работы. Проверим результат.
new_names=['number_of_children', 'days_employed', 'client_age', 
           'education_level', 'education_level_id', 'family_status', 
           'family_status_id', 'gender', 'employment_type', 'debt', 
           'monthly_income', 'credit_purpose']

df.set_axis(new_names, axis='columns', inplace=True)
df.columns

Index(['number_of_children', 'days_employed', 'client_age', 'education_level',
       'education_level_id', 'family_status', 'family_status_id', 'gender',
       'employment_type', 'debt', 'monthly_income', 'credit_purpose'],
      dtype='object')

### 2.2. Обработка пропусков

**Для начала исправим ошибки в данных, а потом заполним пропуски.**

In [12]:
# Вызовем к столбцу 'number_of_children' метод value_counts(), 
# который возвращает уникальные значения и количество их упоминаний.
df['number_of_children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: number_of_children, dtype: int64

In [13]:
# Заменим отрицательные числа на положительные методом replace(), который используется для замены строки другой строкой. 
df['number_of_children'] = df['number_of_children'].replace(-1, 1)  # Заменим -1 на 1.

In [14]:
# 20 детей в семье выглядит странно, посчитаем что произошла ошибка при записи. 
# Заменим 20 детей на 2 ребенка методом replace().
df['number_of_children'] = df['number_of_children'].replace(20, 2)  # Заменим 20 на 2.

In [15]:
# Проверим результат.
df['number_of_children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: number_of_children, dtype: int64

**Рассмотрим значения в переменной 'возраст клиента в годах'.**

In [16]:
# Посчитаем методом value_counts() количество клиентов каждого возраста, по убыванию.
df['client_age'].value_counts(ascending=False)

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: client_age, dtype: int64

In [17]:
# Работать с нулевыми значениями в возрасте не совсем корректно, их 101 человек. Возможно ошибка при записи.
# Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана.
# Для получения среднего значения вызовем метод mean()
df['client_age'].mean()

43.29337979094077

In [18]:
# Заменим нули на среднее методом replace(), который используется для замены строки другой строкой. 
df['client_age'] = df['client_age'].replace(0, 43)  # Заменим 0 на 43.

In [19]:
# Проверим результат.
df['client_age'].value_counts(ascending=False)

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: client_age, dtype: int64

**Разберемся с пропущенными значениями в столбце 'общий трудовой стаж в днях' и 'ежемесячный доход', но для начала исправим ошибки в данных, потому что ошибки могут влиять на медиану и среднее.**

In [20]:
# Вызовем к столбцу 'days_employed' метод value_counts(), который возвращает уникальные значения и количество их упоминаний.
df['days_employed'].value_counts()

-986.93     1
-7026.36    1
-4236.27    1
-6620.40    1
-1238.56    1
           ..
-2849.35    1
-5619.33    1
-448.83     1
-1687.04    1
-582.54     1
Name: days_employed, Length: 19351, dtype: int64

Видим аномалию в данных, это отрицательные значения. Для решения этой проблемы используем метод abs(), который возвращает абсолютное значение заданного числа.

In [21]:
# Вызовем метод abs().
df['days_employed'] = df['days_employed'].abs()
df['days_employed']

0         8437.67
1         4024.80
2         5623.42
3         4124.75
4       340266.07
           ...   
21520     4529.32
21521   343937.40
21522     2113.35
21523     3112.48
21524     1984.51
Name: days_employed, Length: 21525, dtype: float64

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных
значений.

In [22]:
# Суммарное количество пропусков, выявленных методом isnull() в таблице df.
df.isnull().sum()

number_of_children       0
days_employed         2174
client_age               0
education_level          0
education_level_id       0
family_status            0
family_status_id         0
gender                   0
employment_type          0
debt                     0
monthly_income        2174
credit_purpose           0
dtype: int64

In [23]:
# Найдём все строки с пропусками в столбце 'days_employed' и просмотрим первые пять.
# df['days_employed'].isnull() проверяет все строки в столбце 'days_employed' и оставляет только те, в которых есть NaN.
df[df['days_employed'].isnull()].head()

,number_of_children,days_employed,client_age,education_level,education_level_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,credit_purpose
12,0,nan,65,среднее,1,гражданский брак,1,M,пенсионер,0,nan,сыграть свадьбу
26,0,nan,41,среднее,1,женат / замужем,0,M,госслужащий,0,nan,образование
29,0,nan,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,nan,строительство жилой недвижимости
41,0,nan,50,среднее,1,женат / замужем,0,F,госслужащий,0,nan,сделка с подержанным автомобилем
55,0,nan,54,среднее,1,гражданский брак,1,F,пенсионер,1,nan,сыграть свадьбу


In [24]:
# Чтобы сосчитать строки без стажа, вызовем метод count().
df[df['days_employed'].isnull()].count()

number_of_children    2174
days_employed            0
client_age            2174
education_level       2174
education_level_id    2174
family_status         2174
family_status_id      2174
gender                2174
employment_type       2174
debt                  2174
monthly_income           0
credit_purpose        2174
dtype: int64

В 2174 строках из 21525 наблюдений дни стажа пропущены.

In [25]:
# Создаю переменную 'mean_days_employed' и группирую таблицу по столбцу 'client_age',
# чтобы посчитать среднее значение days_employed по возрасту клиента. 
# Потом заменим пропуски на среднее число методом fillna().
mean_days_employed = df.groupby('client_age').agg({'days_employed':'mean'})
 
for client_age in mean_days_employed.index:
    df.loc[df['client_age'] == client_age,'days_employed'] = df.loc[df['client_age'] == client_age,'days_employed']\
      .fillna(mean_days_employed.loc[client_age, 'days_employed'])

mean_days_employed.head()

,days_employed
client_age,
19,633.68
20,684.94
21,709.44
22,2793.32
23,827.31


In [26]:
# Проверим замену.
df.isnull().sum()

number_of_children       0
days_employed            0
client_age               0
education_level          0
education_level_id       0
family_status            0
family_status_id         0
gender                   0
employment_type          0
debt                     0
monthly_income        2174
credit_purpose           0
dtype: int64

In [27]:
# Чтобы сосчитать строки без дохода, вызовем метод count().
df[df['monthly_income'].isnull()].count()

number_of_children    2174
days_employed         2174
client_age            2174
education_level       2174
education_level_id    2174
family_status         2174
family_status_id      2174
gender                2174
employment_type       2174
debt                  2174
monthly_income           0
credit_purpose        2174
dtype: int64

В 2174 строках из 21525 наблюдений доход пропущен.

In [28]:
# Создаю переменную 'mean_monthly_income' и группирую таблицу по столбцу 'employment_type',
# чтобы посчитать среднее значение monthly_income по типу занятости. 
# Потом заменим пропуски на средние число методом fillna().

mean_monthly_income = df.groupby('employment_type').agg({'monthly_income':'mean'})
 
for employment_type in mean_monthly_income.index:
    df.loc[df['employment_type'] == employment_type,'monthly_income'] = df \
      .loc[df['employment_type'] == employment_type,'monthly_income'] \
      .fillna(mean_monthly_income.loc[employment_type, 'monthly_income'])

mean_monthly_income.head()

,monthly_income
employment_type,
безработный,131339.75
в декрете,53829.13
госслужащий,170898.31
компаньон,202417.46
пенсионер,137127.47


После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [29]:
# Проверим замену.
df.isnull().sum()

number_of_children    0
days_employed         0
client_age            0
education_level       0
education_level_id    0
family_status         0
family_status_id      0
gender                0
employment_type       0
debt                  0
monthly_income        0
credit_purpose        0
dtype: int64

In [30]:
# Выведем статистику по ежемесячному доходу методом describe():
df['monthly_income'].describe()

count     21525.00
mean     167395.92
std       97906.95
min       20667.26
25%      107798.17
50%      151931.33
75%      202417.46
max     2265604.03
Name: monthly_income, dtype: float64

На этапе осмотра данных обнаружили большое количество пропущенных значений, пустые значения свидетельствуют о том, что доступна не вся информация. Возможно были проблемы с записью данных. Так как данные отсутствуют и имеем дело с количественными переменными, то заменили пропуски на среднее арифметическое, чтобы примерно оценить типичные значения для каждой выборки, с учетом факторов влияющие на эти значения.

### 2.3. Замена типа данных

Вещественный тип данных необходимо привести к целочисленным значениям. Методом astype() с аргументом ('int') переведем значения с плавающей точкой в целое число.

In [31]:
# Вызовем метод astype() с аргументом 'int'. 
df['days_employed'] = df['days_employed'].astype(np.int64)
df['monthly_income'] = df['monthly_income'].astype(np.int64)

In [32]:
# Также для удобства переведем общий трудовой стаж в года, добавив новый столбец. За один год считается период в 360 дней. 
df['years_employed'] = df['days_employed'] / 360 # значения в днях переведем в года
df['years_employed'] = df['years_employed'].astype(np.int64) # переведем значения с плавающей точкой в целое число

In [33]:
df.info() # Получение общей информации о данных в таблице df. Применим к таблице df метод info().

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   number_of_children  21525 non-null  int64 
 1   days_employed       21525 non-null  int64 
 2   client_age          21525 non-null  int64 
 3   education_level     21525 non-null  object
 4   education_level_id  21525 non-null  int64 
 5   family_status       21525 non-null  object
 6   family_status_id    21525 non-null  int64 
 7   gender              21525 non-null  object
 8   employment_type     21525 non-null  object
 9   debt                21525 non-null  int64 
 10  monthly_income      21525 non-null  int64 
 11  credit_purpose      21525 non-null  object
 12  years_employed      21525 non-null  int64 
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


По данной информации о таблице df видим, что с количеством наблюдений все в порядке их 21525 во всех столбцах. Мы разобрались с отрицательными значениями и изменили их тип, из вещественных сделали целыми, методами, которые описаны в процессе предобработки. 

### 2.4. Преобразование строки в нижний регистр

Уровень образования влияет на получение кредита. Давайте рассмотрим столбец 'education_level'.

In [34]:
# Вызовем к столбцу 'education_level' метод value_counts().
df['education_level'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education_level, dtype: int64

In [35]:
# Приведем строки к нижнему регистру.
df['education_level'] = df['education_level'].str.lower()

# Проверим результат.
df['education_level'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_level, dtype: int64

In [36]:
# Вызовем к столбцу 'education_level_id' метод value_counts().
df['education_level_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_level_id, dtype: int64

**Посмотрим семейное положение, так как оно также влияет на возможность получения кредита. Рассмотрим столбец 'family_status'.**

In [37]:
# Вызовем к столбцу 'family_status' метод value_counts().
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [38]:
# Приведем строки к нижнему регистру.
df['family_status'] = df['family_status'].str.lower()

# Проверим результат.
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [39]:
# Вызовем к столбцу 'family_status_id' метод value_counts().
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [40]:
# Вызовем к столбцу 'employment_type' метод value_counts().
df['employment_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: employment_type, dtype: int64

In [41]:
# Теперь к столбцу 'gender' метод value_counts().
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Наблюдение: в переменной 'пол клиента' присутствует один необычный обьект XNA.

In [42]:
df = df[df['gender'] != 'XNA'] # удалим эту строку

### 2.5. Обработка дубликатов

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [43]:
# Получение суммарного количества дубликатов в таблице df.
df.duplicated().sum()

71

In [44]:
# Удаление всех дубликатов из таблицы df специальным методом.
df = df.drop_duplicates().reset_index(drop=True)

In [45]:
#Проверка на отсутствие.
df.duplicated().sum()

0

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   number_of_children  21453 non-null  int64 
 1   days_employed       21453 non-null  int64 
 2   client_age          21453 non-null  int64 
 3   education_level     21453 non-null  object
 4   education_level_id  21453 non-null  int64 
 5   family_status       21453 non-null  object
 6   family_status_id    21453 non-null  int64 
 7   gender              21453 non-null  object
 8   employment_type     21453 non-null  object
 9   debt                21453 non-null  int64 
 10  monthly_income      21453 non-null  int64 
 11  credit_purpose      21453 non-null  object
 12  years_employed      21453 non-null  int64 
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


Строки-дубликаты могут появиться по разным причинам. Возможно они появились вследствие сбоя в записи данных. Так как дубликаты могут ошибочно повлиять на оценку способности заёмщика вернуть кредит банку, их лучше удалить. Для удаления дубликатов мы использовали метод drop_duplicates(), и соединили в цепочку с вызовом метода reset_index(). Таким образом создали новый DataFrame, где старые индексы превращаются в обычный столбец под названием index, а индексы всех строк снова следуют в естественном порядке. Устранили дубликаты с учётом регистра.

### 2.6. Лемматизация

Выделим леммы в значениях столбца с целями получения кредита. 

In [47]:
# Посмотрим уникальные значения столбца 'credit_purpose', используя метод unique().
unique_credit_purposes = df['credit_purpose'].unique() # сохраным результат в переменной unique_credit_purposes
unique_credit_purposes

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [48]:
m = Mystem()
# Обратите внимание, что Pymystem3 по умолчанию выдает список лемматизированных слов. 
# Поэтому здесь для наглядности результат склеили вызовом метода join(). 
string = '; '.join(unique_credit_purposes) # сохраным результат в переменной string (строка)
string

'покупка жилья; приобретение автомобиля; дополнительное образование; сыграть свадьбу; операции с жильем; образование; на проведение свадьбы; покупка жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; покупка жилой недвижимости; строительство собственной недвижимости; недвижимость; строительство недвижимости; на покупку подержанного автомобиля; на покупку своего автомобиля; операции с коммерческой недвижимостью; строительство жилой недвижимости; жилье; операции со своей недвижимостью; автомобили; заняться образованием; сделка с подержанным автомобилем; получение образования; автомобиль; свадьба; получение дополнительного образования; покупка своего жилья; операции с недвижимостью; получение высшего образования; свой автомобиль; сделка с автомобилем; профильное образование; высшее образование; покупка жилья для сдачи; на покупку автомобиля; ремонт жилью; заняться высшим образованием'

In [49]:
lemmas = m.lemmatize(string) # сохраным результат в переменной lemmas
Counter(lemmas) # посчитаем количество упоминаний, вызовом Counter из модуля collections

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '; ': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

По полученным результатам видно, что самые упомянутые леммы, в значениях столбца с целями получения кредита, это `недвижимость, автомобиль, образование и свадьба. `

### 2.7. Категоризация данных

**Категоризация целей кредита.**

In [50]:
# Напишем функцию для более точного отображения целей кредита.

def credit_purpose_category(credit_purpose):
    if 'образован' in credit_purpose:
        return 'образование'
    if 'авто' in credit_purpose:
        return 'автомобиль'
    if 'свадь' in credit_purpose:
        return 'свадьба'
    if 'жиль' or 'недв' in credit_purpose:
        return 'недвижимость'
    return 'ошибка классификации'

In [51]:
# Осталось создать отдельный столбец с категориями по целям кредита, и в его ячейках записать значения, возвращаемые функцией.
df['credit_purpose_category'] = df['credit_purpose'].apply(credit_purpose_category)

# Выведем статистику по категориям целей кредита методом value_counts():
df['credit_purpose_category'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: credit_purpose_category, dtype: int64

In [52]:
# Проверим, в какую категорию попадёт 'жилье':
credit_purpose_category('жилье')

'недвижимость'

In [53]:
# Проверим, в какую категорию попадёт 'сделка с подержанным автомобилем':
credit_purpose_category('сделка с подержанным автомобилем')

'автомобиль'

In [54]:
# Проверим, в какую категорию попадёт 'сыграть свадьбу':
credit_purpose_category('сыграть свадьбу')

'свадьба'

In [55]:
# Проверим, в какую категорию попадёт 'профильное образование':
credit_purpose_category('профильное образование')

'образование'

**Возраст клиента одно из условий для оформления кредита, он составляет от 18 до 65 лет.** 

Объединим данные в категории, распределим клиентов так:

 - Клиенты младше 18 лет попадают в категорию «дети»;
 - Клиенты от 18 до 65 лет — категория «взрослые»;
 - Клиенты старше 66 лет принадлежат к категории «пенсионеры».

In [56]:
# Запишем правила классификации клиентов как функции.
# На вход функции попадает возраст, а возвращает она категорию клиента. 

def client_age_group(client_age):
    if client_age < 18:
        return 'дети'
    if client_age <= 65:
        return 'взрослые'
    return 'пенсионеры' 

In [57]:
# Осталось создать отдельный столбец с возрастными категориями, и в его ячейках записать значения, возвращаемые функцией.
df['client_age_group'] = df['client_age'].apply(client_age_group)

# Выведем статистику по возрастной группе методом describe():
df['client_age_group'].describe()

count        21453
unique           2
top       взрослые
freq         20751
Name: client_age_group, dtype: object

In [58]:
# Проверим, в какую категорию попадёт четырнадцатилетний клиент:
client_age_group(14)

'дети'

In [59]:
#B какую категорию попадёт сорокалетний клиент:
client_age_group(40)

'взрослые'

In [60]:
#B какую категорию попадёт семидесятилетний клиент:
client_age_group(70)

'пенсионеры'

**Банку важна платежеспособность клиента, давайте рассмотрим, что у нас с ежемесячным доходом.**

In [61]:
# Выведем статистику по ежемесячному доходу методом value_counts():
df['monthly_income'].value_counts()

161380    1071
202417     501
137127     386
170898     145
138071       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: monthly_income, Length: 18608, dtype: int64

Сгруппируем данные в 3 категории по уровню дохода:
- доход меньше 80.000 рублей попадает в категорию «низкий уровень дохода»;
- доход от 80.000 до 180.000 рублей — категория «средний уровень дохода»;
- доход выше 180.000 рублей — категория «высокий уровень дохода».

In [62]:
def level_of_income(monthly_income):
    if monthly_income < 80000:
            return 'низкий уровень дохода'
    if monthly_income < 180000:
            return 'средний уровень дохода'
    return 'высокий уровень дохода'

In [63]:
# Создадим отдельный столбец с категориями уровня дохода, и в его ячейках запишем значения, возвращаемые функцией.
df['level_of_income'] = df['monthly_income'].apply(level_of_income)

# Выведем статистику по категориям уровня дохода методом describe():
df['level_of_income'].describe()

count                      21453
unique                         3
top       средний уровень дохода
freq                       12254
Name: level_of_income, dtype: object

In [64]:
#Проверим, в какую категорию попадёт доход в 45.000 рублей:
level_of_income(45000)

'низкий уровень дохода'

In [65]:
#Проверим, в какую категорию попадёт доход в 130.000 рублей:
level_of_income(130000)

'средний уровень дохода'

In [66]:
#Проверим, в какую категорию попадёт доход в 198.000 рублей:
level_of_income(198000)

'высокий уровень дохода'

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   number_of_children       21453 non-null  int64 
 1   days_employed            21453 non-null  int64 
 2   client_age               21453 non-null  int64 
 3   education_level          21453 non-null  object
 4   education_level_id       21453 non-null  int64 
 5   family_status            21453 non-null  object
 6   family_status_id         21453 non-null  int64 
 7   gender                   21453 non-null  object
 8   employment_type          21453 non-null  object
 9   debt                     21453 non-null  int64 
 10  monthly_income           21453 non-null  int64 
 11  credit_purpose           21453 non-null  object
 12  years_employed           21453 non-null  int64 
 13  credit_purpose_category  21453 non-null  object
 14  client_age_group         21453 non-nul

### 2.8. Вывод

Вывели статистику по возрастной группе клиентов банка, по уровню дохода и целей кредита. Получили количество потенциальных заемщиком, которое составляет 20767 клиентов. И только 17075 клиента со средним и высоким уровнем доходов. С этими данными можно работать.

Проверили все переменные на ошибки. Поработали с пропусками. Решили проблему с аномальными значениями. Удалили дубликаты. Вещественный тип данных привели к целочисленным значениям. Выделили леммы. Объединили данные в категории. Теперь данные годятся для анализа.

## Шаг 3. Ответы на вопросы

### 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [68]:
# Для сравнения наблюдений будем использовать метод свободных таблиц pivot_table().
# Аргументы метода:
    # index — столбец или столбцы, по которым группируют данные 
    # values — значения, по которым мы хотим увидеть сводную таблицу 
    # aggfunc — функция, применяемая к значениям
    
# Результат сохраним в переменной number_of_children_pivot.    
number_of_children_pivot = df.pivot_table(index = 'number_of_children', 
                                          values = 'debt', 
                                          aggfunc = ['count','sum','mean'])

# Создадим в таблице number_of_children_pivot новый столбец 'ratio'. 
# Cохраним в нём значение отношения между наличием детей и возвратом кредита в срок.
# Чем больше значение в столбце 'ratio' (уровень задолжности в %), тем хуже. 

number_of_children_pivot['ratio'] = (number_of_children_pivot['sum'] / number_of_children_pivot['count'] * 100)\
                                                                                            .round(1).astype('str') +'%'
number_of_children_pivot

,count,sum,mean,ratio
,debt,debt,debt,
number_of_children,,,,
0,14090,1063,0.08,7.5%
1,4855,445,0.09,9.2%
2,2128,202,0.09,9.5%
3,330,27,0.08,8.2%
4,41,4,0.10,9.8%
5,9,0,0.00,0.0%


По полученным результатам видно, что существует зависимость между наличием детей и возвратом кредита в срок. Клиенты с детьми возвращают кредит в срок хуже, чем без детей, возможно это связано с тем, что наличие детей налагает на клиента повышенные финансовые обязательства, тем самым понижая его платежеспособность.

### 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [69]:
# Используем метод pivot_table(), результат сохраним в переменной family_status_pivot.
family_status_pivot = df.pivot_table(index = 'family_status', 
                                     values = 'debt', 
                                     aggfunc = ['count','sum','mean'])

family_status_pivot['ratio'] = (family_status_pivot['sum'] / family_status_pivot['count'] * 100)\
                                                                            .round(1).astype('str') +'%'
family_status_pivot

,count,sum,mean,ratio
,debt,debt,debt,
family_status,,,,
в разводе,1195,85,0.07,7.1%
вдовец / вдова,959,63,0.07,6.6%
гражданский брак,4150,388,0.09,9.3%
женат / замужем,12339,931,0.08,7.5%
не женат / не замужем,2810,274,0.10,9.8%


Самые высокие показатели у клиентов которые состоят в гражданском браке и у тех кто вовсе в нём не состоит, они хуже всех возвращают кредиты в срок. 

### 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [70]:
# Используем метод pivot_table(), результат сохраним в переменной monthly_income_pivot.
monthly_income_pivot = df.pivot_table(index = 'level_of_income', 
                                      values = 'debt', 
                                      aggfunc = ['count','sum','mean'])

monthly_income_pivot['ratio'] = (monthly_income_pivot['sum'] / monthly_income_pivot['count'] * 100)\
                                                                                            .round(1).astype('str') +'%'
monthly_income_pivot

,count,sum,mean,ratio
,debt,debt,debt,
level_of_income,,,,
высокий уровень дохода,6923,508,0.07,7.3%
низкий уровень дохода,2276,174,0.08,7.6%
средний уровень дохода,12254,1059,0.09,8.6%


Самый высокий показатель у клиентов со средним уровень дохода, они хуже всех возвращают кредиты в срок.

### 3.4. Как разные цели кредита влияют на его возврат в срок?

In [71]:
# Используем метод pivot_table(), результат сохраним в переменной credit_purpose_pivot.
credit_purpose_pivot = df.pivot_table(index = 'credit_purpose_category', 
                                      values = 'debt', 
                                      aggfunc = ['count','sum','mean'])

credit_purpose_pivot['ratio'] = (credit_purpose_pivot['sum'] /credit_purpose_pivot['count'] * 100)\
                                                                                            .round(1).astype('str') +'%'
credit_purpose_pivot

,count,sum,mean,ratio
,debt,debt,debt,
credit_purpose_category,,,,
автомобиль,4306,403,0.09,9.4%
недвижимость,10810,782,0.07,7.2%
образование,4013,370,0.09,9.2%
свадьба,2324,186,0.08,8.0%


По полученным результатам видно, что клиенты с целями кредита недвижимость и свадьба лучше возвращают кредит в срок. 

## Шаг 4. Общий вывод исследования

Подводя итоги, мы видим, что существует зависимость между наличием детей, семейного положения, уровня дохода и целей кредита, с возвратом кредита в срок.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**
Банку важна платежеспособность клиента. Наличие детей налагает на клиента повышенные финансовые обязательства и, по факту, снижает его платежеспособность. Самый низкий уровень задолженности у бездетных, он составляет 7,5%. И самый высокий уровень задолженности у клиентов с 4 детьми, 9,7%.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**
Видим, что больший шанс оформить и получить кредит будет у того потенциального заемщика, который состоит в браке или вдовец. Самый низкий уровень задолженности у вдовцов / вдов, он составляет 6,5%. У клиентов, состоящих в браке, 7,5%.  И самый высокий уровень задолженности у клиентов, которые состоят в гражданском браке и у тех, кто вовсе в нём не состоит, 9,7%, они хуже всех возвращают кредиты в срок.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**
По полученным результатам видно, что клиенты с высоким уровнем доходов лучше всех справляются с выплатами по кредиту, у них уровень задолженности составляет 7%. Уровень задолженности у клиентов со средним уровнем дохода 8,6%, они хуже всех возвращают кредиты в срок.

**Как разные цели кредита влияют на его возврат в срок?**
Клиенты с целями кредита недвижимость и свадьба лучше возвращают кредит в срок, их уровень задорности 7%.
Хуже возвращают кредит в срок, клиенты цели кредита которых автомобиль и образование, у них уровень задолженности 9%. 

**Рекомендация:**
 - Исходя из результатов исследования, вероятнее, кредит выдадут человеку женатому, но без детей, с высоким уровнем дохода, цели которого недвижимость.
 - Особое внимание следует уделить клиентам не женатым или состоящих в гражданском браке, с детьми, со средним уровнем дохода, с целями кредита автомобиль, у них самые плохие показатели. У таких клиентов риски по невозврату кредитов выше.